# LUCAS COPERNICUS
# Creation of the random forest models for the Eurocropmap
## Land cover mask

In [1]:
# JEODPP
data_path='/eos/jeodpp/data/projects/REFOCUS/data/S1_GS/all-10days/Map_v7/'
project_path='/eos/jeodpp/data/projects/REFOCUS/classification/'
path_pol = '/eos/jeodpp/data/projects/REFOCUS/data/polygons/v7'
results='/eos/jeodpp/data/projects/REFOCUS/classification/'

local='/eos/jeodpp/home/users/verheas/data/LUCAS/v7/'

# RAF LOCAL
# data_path='/data/LUCAS-cop-single-pixel'
# project_path='/data/Dropbox/JRC/LANDSENSE/CASE-STUDY-8-LUCAS-COPERNICUS-CLASSIF/'

In [2]:
#import 
import pandas as pd
from pandas import Series,DataFrame
#import geopandas as gdp
import csv
import numpy as np
import time
import sklearn
import scipy
import matplotlib.pyplot as plt
import glob
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
#pd.show_versions()

## The parameters

In [3]:
##################################Parameters##################################################
##################################Parameters##################################################
#classes - stored in a table 'legend-lucas-all'
table_class=pd.read_csv(os.path.join(project_path,'table/legend-lucas-all-v7.csv'),dtype=pd.Int64Dtype())

classes_L1=list(table_class['classes_L1'].dropna()) 
classes_L2=list(table_class['classes_L2'].dropna())

#Biome selection

biome_1=[1]
biome_2=[2]
biome_3=[3]
biome_4=[4]

#level
level_1='level_1'
level_2='level_2'

## Load the data

In [4]:
## Load the data
#1) load the S1 10 days extracted values in GEE for all polygons

pd_lucas= pd.read_csv(os.path.join(data_path,'S1_point_allV7_10days_10m_1Jan-31Dec_EU_ratio-db.csv'),dtype={'level_1':int,'level_2':int})
print('pd_lucas',pd_lucas.shape)

#concatenate all the data in one dataframe
#group cropland, grassland and bareland 
#number of pixels per class
print(pd_lucas.level_1.value_counts())
print(pd_lucas.level_2.value_counts())
pd_lucas.head()

#number of pixels per class
#pd_lucas.LC1_COD.value_counts()
#pd_lucas.head()
pd_lucas.columns

##############1.2 Load the shapefile with the polygons - useful to split the polygons in training and test dataset for the accuracy ######################
# load csv with of the polygons
#2)load csv with the polygons for the split test/validation
lucas_polygons = pd.read_csv(os.path.join(path_pol,'LUCAS_2018_Copernicus_attributes_cropmap_level1-2.csv'))
lucas_polygons.head()

pd_lucas (2956889, 116)
300    1216530
200    1000318
500     732964
600       3856
100       3221
Name: level_1, dtype: int64
300    1216530
500     732964
211     290116
213     142886
216     125644
232      64452
290      63609
250      59053
214      35215
231      34577
240      34067
215      34005
212      28825
222      23174
218      19274
221      15815
230      12000
233       7094
223       4920
219       4724
600       3856
100       3221
217        868
Name: level_2, dtype: int64


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (27,30,33,104,105) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,POINT_ID,NUTS0,NUTS1,NUTS2,NUTS3,TH_LAT,TH_LONG,OFFICE_PI,EX_ANTE,...,LU2_LABEL,LU1_TYPE_LABEL,LU2_TYPE_LABEL,CPRN_LC_LABEL,CPRN_LC_SAME_LC1,LUCAS_CORE_INTERSECT,COPERNICUS_CLEANED,stratum,level_2,level_1
0,0,34562080,ES,ES2,ES24,ES243,41.288386,-0.319428,0,0,...,Not relevant,NaN,NaN,Other bare soil,True,True,True,2,290,200
1,1,31243520,IE,IE0,IE04,IE042,53.422977,-8.226052,0,0,...,Not relevant,NaN,NaN,Spontaneously re-vegetated surfaces,True,True,True,1,500,500
2,2,33661774,ES,ES5,ES52,ES521,38.434388,-0.905705,0,0,...,Not relevant,NaN,NaN,Permanent crops: fruit trees,True,True,True,2,300,300
3,3,28922250,ES,ES1,ES11,ES113,41.867145,-7.320304,0,0,...,Not relevant,NaN,NaN,Shrubland with sparse tree cover,True,True,True,2,300,300
4,4,35082906,FR,FRD,FRD1,FRD12,48.715190,-1.092190,0,0,...,Not relevant,NaN,NaN,Grassland without tree/shrub cover,True,True,True,1,500,500


# Biome 1 - parameters

In [8]:
###################################Choose parameters for this run #############################################
#Biome
biome=biome_1
print('biome',biome)

#level
level=level_1
print('level',level)
#crop - level 2, from the table we load only the crop type classes
classes=classes_L1
print('level',classes)

#Split for the train/test dataset - we run it with all the polygons
#split_test = 0

biome [1]
level level_1
level [100, 200, 300, 500, 600]


## Biome 1 - Prepare the data

In [9]:
##############################################################
#### 2) Prepare the data for the classification ##############
##############################################################

#############2.1 Select level of work and classes
#copy values in a new column 'Classif' that we will use in the rest of the script
pd_lucas['Classif']=pd_lucas[level]
print(pd_lucas.shape)

pd_lucas_i=pd_lucas[pd_lucas.Classif.isin(classes_L1)]

#############2.2 Select the biome
#select biome
pd_lucas_b=pd_lucas_i[pd_lucas_i.stratum.isin(biome)]
print('pd_lucas_b',pd_lucas_b.Classif.value_counts())
print('pd_lucas_b',pd_lucas.Classif.value_counts())
print(pd_lucas_b.groupby('POINT_ID').apply(min).stratum.value_counts())

#############2.3 Select the data inputs for the classification
## we use all the polygons, therefore there are no test dataset

X_train=pd_lucas_b.filter(regex='(((?<![\w\d])VH_)|((?<![\w\d])VV_))(20180[1-7])')
y_train=pd_lucas_b['Classif']

print('X_train head',X_train.head())
print('X_train shape',X_train.shape)

print('y_train shape',y_train.shape)
print('y_train count',y_train.value_counts())


(2956889, 117)
pd_lucas_b 300    932467
200    813124
500    624627
600      2488
100      2449
Name: Classif, dtype: int64
pd_lucas_b 300    1216530
200    1000318
500     732964
600       3856
100       3221
Name: Classif, dtype: int64
1    43898
Name: stratum, dtype: int64
X_train head     VH_20180101  VH_20180111  VH_20180121  VH_20180131  VH_20180210  \
16   -15.841164   -16.101488   -16.373434   -15.005488   -17.550755   
17   -14.036120   -14.616349   -14.647615   -14.757492   -15.923586   
18   -14.036120   -15.017238   -13.691375   -13.207221   -15.077690   
19   -10.934003   -10.906717   -10.008709   -12.063789   -12.230721   
20   -11.434500   -11.976395   -11.637123   -12.429725   -12.179104   

    VH_20180220  VH_20180302  VH_20180312  VH_20180322  VH_20180401  ...  \
16   -16.679602   -17.511936   -17.628834   -16.359150   -13.442070  ...   
17   -16.752005   -17.511936   -17.170132   -15.328428   -11.951139  ...   
18   -16.350103   -16.916924   -15.727412   -15.328428 

In [10]:
rfc = RandomForestClassifier(bootstrap=0, criterion='gini', max_depth=None, max_features='sqrt', 
                             min_samples_leaf=4, min_samples_split=3, n_estimators=800, n_jobs=40)
rfc.fit(X_train.values, y_train.values)
pickle.dump(rfc, open(os.path.join(project_path,'RFmodel_LUCAS_'+str(biome)+'_'+str(level)+'_all-polygons_janv-jul2018_15122020b'), 'wb'))


OSError: [Errno 30] Read-only file system: '/eos/jeodpp/data/projects/REFOCUS/classification/RFmodel_LUCAS_[1]_level_1_all-polygons_janv-jul2018_15122020b'

## Biome 1 - Run a RandomizedSearchCV

In [3]:
#Grid Search for a set of parameters - without scaler - no need of pipeline?
#there is also the randomized search CV that will not run all the possibilities - we can use to get a first idea
t = time.time()
Method = 'Random Forest'
print('Method:', Method)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.pipeline import Pipeline

param_grid = { 
    'RFclf__n_estimators': [150,180,210,240,270,300,500,800,900,1000,1100,1200], #The number of trees in the forest.
    'RFclf__max_features': ['auto', 'sqrt', 'log2'], #The number of features to consider when looking for the best split:
    'RFclf__max_depth': [4,5,6,8,12,None],#The maximum depth of the tree
    'RFclf__min_samples_leaf': [1,2,3,4,8,10,12],#The minimum number of samples in newly created leaves.
    'RFclf__bootstrap': [0, 1],#Whether bootstrap samples are used when building trees.
    'RFclf__min_samples_split': [3, 5, 7], #The minimum number of samples required to split an internal node
    'RFclf__criterion': ['gini', 'entropy']#The function to measure the quality of a split
}
#Scaler = preprocessing.StandardScaler()
#Scaler.fit(TrF)
Pipeline = Pipeline([('RFclf', RandomForestClassifier())])
#rfc = GridSearchCV(estimator=Pipeline, param_grid=param_grid,cv=3, n_jobs=100, verbose=1)
rfc = RandomizedSearchCV(estimator=Pipeline, param_distributions =param_grid, n_iter=100,cv=3, n_jobs=20, verbose=1)
#RandomizedSearchCV
#rfc.fit(Scaler.transform(TrF), TrC)
rfc.fit(X_train, y_train)
print('Elapsed time for training: %.02f sec' % (time.time() - t))


pickle.dump(rfc, open('RFmodel_LUCAS_'+str(biome)+'_'+str(level)+'_all-polygons_janv-jul2018_15122020best', 'wb'))

print(rfc.best_estimator_.named_steps['RFclf'])

#best_model=rfc.best_estimator_.named_steps['RFclf']

#best model
#print('best estimator',best_model)

#pickle.dump(best_model, open('RFmodel_LUCAS_'+str(biome)+'_level1-mask_all-polygons_janv-jul2018_160920-best', 'wb'))

NameError: name 'time' is not defined

In [ ]:
print(rfc.best_estimator_.named_steps['RFclf'])

In [ ]:
#split the data
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
phat = clf.predict_proba(X_test)[:,1]

# Biome 2 - parameters

In [8]:
###################################Choose parameters for this run #############################################
#Biome
biome=biome_2
print('biome',biome)

#level
level=level_1
print('level',level)
#crop - level 2, from the table we load only the crop type classes
classes=classes_L1
print('level',classes)

#Split for the train/test dataset - we run it with all the polygons
#split_test = 0

biome [2]
level level_1
level [100, 200, 300, 500, 600]


## Biome 2 - Prepare the data

In [9]:
##############################################################
#### 2) Prepare the data for the classification ##############
##############################################################

#############2.1 Select level of work and classes
#copy values in a new column 'Classif' that we will use in the rest of the script
pd_lucas['Classif']=pd_lucas[level]
print(pd_lucas.shape)

pd_lucas_i=pd_lucas[pd_lucas.Classif.isin(classes)]

#############2.2 Select the biome
#select biome
pd_lucas_b=pd_lucas_i[pd_lucas_i.stratum.isin(biome)]
print('pd_lucas_b',pd_lucas_b.Classif.value_counts())
print('pd_lucas_b',pd_lucas.Classif.value_counts())
print(pd_lucas_b.groupby('POINT_ID').apply(min).stratum.value_counts())

#############2.3 Select the data inputs for the classification
## we use all the polygons, therefore there are no test dataset

X_train=pd_lucas_b.filter(regex='(((?<![\w\d])VH_)|((?<![\w\d])VV_))(20180[1-7])')
y_train=pd_lucas_b['Classif']

print('X_train head',X_train.head())
print('X_train shape',X_train.shape)

print('y_train shape',y_train.shape)
print('y_train count',y_train.value_counts())


(2956889, 117)
pd_lucas_b 300    284063
200    187194
500    108337
600      1368
100       772
Name: Classif, dtype: int64
pd_lucas_b 300    1216530
200    1000318
500     732964
600       3856
100       3221
Name: Classif, dtype: int64
2    14280
Name: stratum, dtype: int64
X_train head    VH_20180101  VH_20180111  VH_20180121  VH_20180131  VH_20180210  \
0   -16.954021   -16.242271   -15.693783   -18.552937   -17.400238   
1   -17.597748   -18.795210   -18.761711   -19.685406   -18.706253   
2   -16.457218   -18.847889   -17.879326   -19.397968   -17.177315   
3   -15.408525   -17.031052   -15.888536   -17.946339   -15.941470   
4   -12.962501   -14.945486   -13.482009   -16.617935   -15.778416   

   VH_20180220  VH_20180302  VH_20180312  VH_20180322  VH_20180401  ...  \
0   -16.954105   -16.281940   -15.294356   -16.696960   -17.772724  ...   
1   -16.822638   -17.532448   -19.449732   -18.291378   -19.510078  ...   
2   -16.013180   -17.093063   -17.248064   -17.229742   -17.3037

## Biome 2 - Run a RandomizedSearchCV

In [ ]:
#Grid Search for a set of parameters - without scaler - no need of pipeline?
#there is also the randomized search CV that will not run all the possibilities - we can use to get a first idea
t = time.time()
Method = 'Random Forest'
print('Method:', Method)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.pipeline import Pipeline

param_grid = { 
    'RFclf__n_estimators': [150,180,210,240,270,300,500,800,900,1000,1100,1200],
    'RFclf__max_features': ['auto', 'sqrt', 'log2'],
    'RFclf__max_depth': [4,5,6,8,12,None],
    'RFclf__min_samples_leaf': [1,2,3,4,8,10,12],
    'RFclf__bootstrap': [0, 1],
    'RFclf__min_samples_split': [3, 5, 7],
    'RFclf__criterion': ['gini', 'entropy']
}
#Scaler = preprocessing.StandardScaler()
#Scaler.fit(TrF)
Pipeline = Pipeline([('RFclf', RandomForestClassifier())])
#rfc = GridSearchCV(estimator=Pipeline, param_grid=param_grid,cv=3, n_jobs=100, verbose=1)
rfc = RandomizedSearchCV(estimator=Pipeline, param_distributions =param_grid, n_iter=100,cv=3, n_jobs=38, verbose=1)
#RandomizedSearchCV
#rfc.fit(Scaler.transform(TrF), TrC)
rfc.fit(X_train, y_train)
print('Elapsed time for training: %.02f sec' % (time.time() - t))


pickle.dump(rfc, open('RFmodel_LUCAS_'+str(biome)+'_'+str(level)+'_all-polygons_janv-jul2018_15122020best', 'wb'))

print(rfc.best_estimator_.named_steps['RFclf'])

#best_model=rfc.best_estimator_.named_steps['RFclf']

#best model
#print('best estimator',best_model)

#pickle.dump(best_model, open('RFmodel_LUCAS_'+str(biome)+'_level1-mask_all-polygons_janv-jul2018_160920-best', 'wb'))

Method: Random Forest
Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=38)]: Using backend LokyBackend with 38 concurrent workers.


# Biome 3 - parameters

In [ ]:
###################################Choose parameters#############################################
classes=classes_L1

#reclassification
classes_in=classes_in_L1
print ('classes_in',classes_in)

#classes_remap=classes_remap_L1
#print ('classes_remap',classes_remap)

#classes for the classification and biome/no biome differentiation if needed
classes_B=classes_L1_B
print ('classes_B',classes_B)

classes_NB=classes_L1_NB
print ('classes_NB',classes_NB)

#summary of the classses used for the classification
classes_classif=classes_classif_L1
print ('classes_classif',classes_classif)
classes_classif_simplify=classes_classif_L1_simplify
print ('classes_classif_simplify',classes_classif_simplify)

#Biome
biome=biome_3
print('biome',biome)

#level
level=level_1
print('level',level)

## Biome 3 - Prepare the data

In [ ]:
##############################################################
#### 2) Prepare the data for the classification ##############
##############################################################

#############2.1 Reclassify
#copy values in a new column 'Classif' that we will use in the rest of the script
pd_lucas_i=pd_lucas

pd_lucas_i['Classif']=pd_lucas_i[level]
pd_lucas_i.head()
#print(pd_lucas.Classif.value_counts())

#reclassification of the classes according to the level
#pd_lucas.Classif=pd_lucas.Classif.replace(classes_in,
#                                            classes_remap)
print(pd_lucas_i.shape)
#print(pd_lucas.Classif.value_counts())

#############2.2 Select the biome
#create a column with the value of the class + the biome (add the biome after the class)
#for the level-1 class, the 
pd_lucas_biome=pd_lucas_i[pd_lucas_i.Classif.isin(classes_B)]
pd_lucas_nobiome=pd_lucas_i[pd_lucas_i.Classif.isin(classes_NB)]

pd_lucas_biome['ClassifB']=pd_lucas_biome['Classif'].astype(str) + pd_lucas_biome['stratum'].astype(str)
pd_lucas_nobiome['ClassifB']=pd_lucas_nobiome['Classif'].astype(str) + '0'

#select biome
pd_lucas_biome=pd_lucas_biome[pd_lucas_biome.stratum.isin(biome)]
pd_lucas_b=pd_lucas_biome.append(pd_lucas_nobiome)

#print(pd_lucas_b.head())
print('pd_lucas_b',pd_lucas_b.ClassifB.value_counts())
print('pd_lucas_b',pd_lucas_b.Classif.value_counts())

#############2.3 Select the data inputs for the classification
## we use all the polygons, therefore there are no test dataset

X_train=pd_lucas_b.filter(regex='(((?<![\w\d])VH_)|((?<![\w\d])VV_))(20180[1-7])')
y_train=pd_lucas_b['Classif']

print('X_train head',X_train.head())
print('X_train shape',X_train.shape)

print('y_train shape',y_train.shape)
print('y_train count',y_train.value_counts())


## Biome 3 - Run a RandomizedSearchCV

In [ ]:
#Grid Search for a set of parameters - without scaler - no need of pipeline?
#there is also the randomized search CV that will not run all the possibilities - we can use to get a first idea
t = time.time()
Method = 'Random Forest'
print('Method:', Method)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.pipeline import Pipeline

param_grid = { 
    'RFclf__n_estimators': [50,75,100,130,150,180,210,240,270],
    'RFclf__max_features': ['auto', 'sqrt', 'log2'],
    'RFclf__max_depth': [4,5,6,8,12,None],
    'RFclf__min_samples_leaf': [1,2,3,4,8,10,12],
    'RFclf__bootstrap': [0, 1],
    'RFclf__min_samples_split': [3, 5, 7],
    'RFclf__criterion': ['gini', 'entropy']
}
#Scaler = preprocessing.StandardScaler()
#Scaler.fit(TrF)
Pipeline = Pipeline([('RFclf', RandomForestClassifier())])
#rfc = GridSearchCV(estimator=Pipeline, param_grid=param_grid,cv=3, n_jobs=100, verbose=1)
rfc = RandomizedSearchCV(estimator=Pipeline, param_distributions =param_grid, n_iter=100,cv=3, n_jobs=38, verbose=1)
#RandomizedSearchCV
#rfc.fit(Scaler.transform(TrF), TrC)
rfc.fit(X_train, y_train)
print('Elapsed time for training: %.02f sec' % (time.time() - t))


pickle.dump(rfc, open('RFmodel_LUCAS_'+str(biome)+'_'+str(level)+'_all-polygons_janv-jul2018_17092020-bis', 'wb'))

print(rfc.best_estimator_.named_steps['RFclf'])

#best_model=rfc.best_estimator_.named_steps['RFclf']

#best model
#print('best estimator',best_model)

#pickle.dump(best_model, open('RFmodel_LUCAS_'+str(biome)+'_level1-mask_all-polygons_janv-jul2018_160920-best', 'wb'))

# Biome 4 - parameters

In [ ]:
###################################Choose parameters#############################################
classes=classes_L1

#reclassification
classes_in=classes_in_L1
print ('classes_in',classes_in)

#classes_remap=classes_remap_L1
#print ('classes_remap',classes_remap)

#classes for the classification and biome/no biome differentiation if needed
classes_B=classes_L1_B
print ('classes_B',classes_B)

classes_NB=classes_L1_NB
print ('classes_NB',classes_NB)

#summary of the classses used for the classification
classes_classif=classes_classif_L1
print ('classes_classif',classes_classif)
classes_classif_simplify=classes_classif_L1_simplify
print ('classes_classif_simplify',classes_classif_simplify)

#Biome
biome=biome_4
print('biome',biome)

#level
level=level_1
print('level',level)

## Biome 4 - Prepare the data

In [ ]:
##############################################################
#### 2) Prepare the data for the classification ##############
##############################################################

#############2.1 Reclassify
#copy values in a new column 'Classif' that we will use in the rest of the script
pd_lucas_i=pd_lucas

pd_lucas_i['Classif']=pd_lucas_i[level]
pd_lucas_i.head()
#print(pd_lucas.Classif.value_counts())

#reclassification of the classes according to the level
#pd_lucas.Classif=pd_lucas.Classif.replace(classes_in,
#                                            classes_remap)
print(pd_lucas_i.shape)
#print(pd_lucas.Classif.value_counts())

#############2.2 Select the biome
#create a column with the value of the class + the biome (add the biome after the class)
#for the level-1 class, the 
pd_lucas_biome=pd_lucas_i[pd_lucas_i.Classif.isin(classes_B)]
pd_lucas_nobiome=pd_lucas_i[pd_lucas_i.Classif.isin(classes_NB)]

pd_lucas_biome['ClassifB']=pd_lucas_biome['Classif'].astype(str) + pd_lucas_biome['stratum'].astype(str)
pd_lucas_nobiome['ClassifB']=pd_lucas_nobiome['Classif'].astype(str) + '0'

#select biome
pd_lucas_biome=pd_lucas_biome[pd_lucas_biome.stratum.isin(biome)]
pd_lucas_b=pd_lucas_biome.append(pd_lucas_nobiome)

#print(pd_lucas_b.head())
print('pd_lucas_b',pd_lucas_b.ClassifB.value_counts())
print('pd_lucas_b',pd_lucas_b.Classif.value_counts())

#############2.3 Select the data inputs for the classification
## we use all the polygons, therefore there are no test dataset

X_train=pd_lucas_b.filter(regex='(((?<![\w\d])VH_)|((?<![\w\d])VV_))(20180[1-7])')
y_train=pd_lucas_b['Classif']

print('X_train head',X_train.head())
print('X_train shape',X_train.shape)

print('y_train shape',y_train.shape)
print('y_train count',y_train.value_counts())


## Biome 4 - Run a RandomizedSearchCV

In [ ]:
#Grid Search for a set of parameters - without scaler - no need of pipeline?
#there is also the randomized search CV that will not run all the possibilities - we can use to get a first idea
t = time.time()
Method = 'Random Forest'
print('Method:', Method)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.pipeline import Pipeline

param_grid = { 
    'RFclf__n_estimators': [50,75,100,130,150,180,210,240,270],
    'RFclf__max_features': ['auto', 'sqrt', 'log2'],
    'RFclf__max_depth': [4,5,6,8,12,None],
    'RFclf__min_samples_leaf': [1,2,3,4,8,10,12],
    'RFclf__bootstrap': [0, 1],
    'RFclf__min_samples_split': [3, 5, 7],
    'RFclf__criterion': ['gini', 'entropy']
}
#Scaler = preprocessing.StandardScaler()
#Scaler.fit(TrF)
Pipeline = Pipeline([('RFclf', RandomForestClassifier())])
#rfc = GridSearchCV(estimator=Pipeline, param_grid=param_grid,cv=3, n_jobs=100, verbose=1)
rfc = RandomizedSearchCV(estimator=Pipeline, param_distributions =param_grid, n_iter=100,cv=3, n_jobs=38, verbose=1)
#RandomizedSearchCV
#rfc.fit(Scaler.transform(TrF), TrC)
rfc.fit(X_train, y_train)
print('Elapsed time for training: %.02f sec' % (time.time() - t))


pickle.dump(rfc, open('RFmodel_LUCAS_'+str(biome)+'_'+str(level)+'_all-polygons_janv-jul2018_17092020-bis', 'wb'))

print(rfc.best_estimator_.named_steps['RFclf'])

#best_model=rfc.best_estimator_.named_steps['RFclf']

#best model
#print('best estimator',best_model)

#pickle.dump(best_model, open('RFmodel_LUCAS_'+str(biome)+'_level1-mask_all-polygons_janv-jul2018_160920-best', 'wb'))

In [ ]:
#Grid Search for a set of parameters - takes time 
#there is also the randomized search CV that will not run all the possibilities - we can use to get a first idea
t = time.time()
Method = 'Random Forest'
print('Method:', Method)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.pipeline import Pipeline

param_grid = { 
    'RFclf__n_estimators': [50,75,100,130,150,180],
    'RFclf__max_features': ['auto', 'sqrt', 'log2'],
    'RFclf__max_depth': [4,5,6,8,12,None],
    'RFclf__min_samples_leaf': [1,2,3,4,8,10,12],
    'RFclf__bootstrap': [0, 1],
    'RFclf__min_samples_split': [3, 5, 7],
    'RFclf__criterion': ['gini', 'entropy']
}
#Scaler = preprocessing.StandardScaler()
#Scaler.fit(TrF)
Pipeline = Pipeline([('Scaler', preprocessing.StandardScaler()), ('RFclf', RandomForestClassifier())])
#rfc = GridSearchCV(estimator=Pipeline, param_grid=param_grid,cv=3, n_jobs=100, verbose=1)
rfc = RandomizedSearchCV(estimator=Pipeline, param_distributions =param_grid, n_iter=100,cv=3, n_jobs=38, verbose=1)
#RandomizedSearchCV
#rfc.fit(Scaler.transform(TrF), TrC)
rfc.fit(X_train, y_train)
print('Elapsed time for training: %.02f sec' % (time.time() - t))


pickle.dump(rfc, open('RFmodel_LUCAS_'+str(biome)+str(level)+'_all-polygons_janv-jul2018_17092020', 'wb'))

print(rfc.best_estimator_.named_steps['RFclf'])

#best model
#print('best estimator',best_model)

#pickle.dump(best_model, open('RFmodel_LUCAS_'+str(biome)+'_level1-mask_all-polygons_janv-jul2018_160920-best', 'wb'))

In [17]:
classifier = rfc.best_estimator_.named_steps['RFclf']
print (classifier)
pickle.dump(classifier, open('RFmodel_LUCAS_'+str(biome)+'_level1-mask_all-polygons_janv-jul2018_160920-best', 'wb'))

RandomForestClassifier(bootstrap=0, class_weight=None, criterion='gini',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=7,
            min_weight_fraction_leaf=0.0, n_estimators=180, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [16]:
print(rfc.best_estimator_.steps[1][1])


RandomForestClassifier(bootstrap=0, class_weight=None, criterion='gini',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=7,
            min_weight_fraction_leaf=0.0, n_estimators=180, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


## TEST FOR ONE SET OF PARAMETERS

In [9]:
#TEST WITH RANDOM PARAMETERS TO TEST FEATURES IMPORTANCE
#July
#split test and validation
X_trainP,X_testP,y_trainP,y_testP  = train_test_split(X_train,y_train, test_size=0.25,random_state=5)#,stratify=[200,300,500,600,100])

t = time.time()
Method = 'Random Forest'
print('Method:', Method)
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(bootstrap=0, criterion='gini', max_depth=None, max_features='sqrt', 
                             min_samples_leaf=4, min_samples_split=3, n_estimators=150, n_jobs=40)

rfc.fit(X_trainP.values, y_trainP.values)

print('Elapsed time for training: %.02f sec' % (time.time() - t))
#pickle.dump(clf, open('RFmodel_1x1_scenario2_test', 'wb'))


#accuracy
y_test_pred=rfc.predict(X_testP)      
y_test_pred_s=pd.Series(y_test_pred, dtype='float')
        
#to calculate accuracy, go back to array    
accuracy = 100.0*(y_testP.array == y_test_pred_s.array).sum()/X_testP.shape[0]
print('Accuracy is :' + str(round(accuracy,2)))

confusion_mat=confusion_matrix(y_testP,y_test_pred_s,labels=list([200,300,500,600,100]))
print (confusion_mat)
import pickle
pickle.dump(rfc, open('RFmodel_LUCAS_'+str(biome)+'_level1-mask_all-polygons_janv-juil2018bis', 'wb'))

Method: Random Forest
Elapsed time for training: 141.43 sec
Accuracy is :94.93
[[89168   221  2963     0     0]
 [  605 76324  2364     0     1]
 [ 3819  1342 55756     0     1]
 [  135    66   119    56     0]
 [   38    78    70     0    42]]


In [29]:
print(y_train)

0        200
1        200
2        200
3        200
4        200
5        200
6        200
7        200
8        200
9        200
10       200
11       200
12       200
13       200
14       200
15       200
16       200
17       200
18       200
19       200
20       200
21       200
22       200
23       200
24       200
25       200
26       200
27       200
28       200
29       200
        ... 
30244    600
30245    600
30246    600
30247    600
30248    600
30249    600
30250    600
30251    600
30252    600
30253    600
30254    600
30255    600
30256    600
30257    600
30258    600
30259    600
30260    600
30261    600
30262    600
30263    600
30264    600
30265    600
30266    600
30267    600
30268    600
30269    600
30270    600
30271    600
30272    600
30273    600
Name: Classif, Length: 932671, dtype: int64


In [27]:
print(classes_L1)

[100, 200, 510, 300, 400, 500, 600, 520, 530]


In [9]:
pickle.dump(rfc, open(local+'RFmodel_LUCAS_'+str(biome)+'_level1-mask_all-polygons_janv-aug2018_24042020', 'wb'))

OSError: [Errno 30] Read-only file system: '/eos/jeodpp/data/projects/REFOCUS/classification/RFmodel_LUCAS_[1]_level1-mask_all-polygons_janv-aug2018_24042020'

In [8]:
Model=rfc
print (Model)

importances = Model.feature_importances_

std = np.std([tree.feature_importances_ for tree in Model], axis=0)

indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(44):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]),X_train.columns[indices[f]])


RandomForestClassifier(bootstrap=0, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=40,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Feature ranking:
1. feature 12 (0.081311) VH_20180501
2. feature 13 (0.068887) VH_20180511
3. feature 11 (0.064408) VH_20180421
4. feature 22 (0.056444) VV_20180101
5. feature 10 (0.051337) VH_20180411
6. feature 21 (0.049486) VH_20180730
7. feature 14 (0.044355) VH_20180521
8. feature 9 (0.042007) VH_20180401
9. feature 41 (0.029622) VV_20180710
10. feature 20 (0.028934) VH_20180720
11. feature 5 (0.028927) VH_20180220
12. feature 34 (0.028375) VV_20180501
13. feature 43 (0.027530) VV_20180730
14. feature 19 (0.024861) VH_20180710
15. feature 15 (0.021888) VH_20180531
16. 

In [8]:
# Features importances for the best RF model from the Grid CV
Model=rfc
print(Model)
importances = Model.best_estimator_.steps[1][1].feature_importances_

std = np.std([tree.feature_importances_ for tree in Model.best_estimator_.steps[1][1]], axis=0)

indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(44):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]),X_train.columns[indices[f]])


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('Scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('RFclf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
   ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params=None, iid='warn', n_iter=100, n_jobs=38,
          param_distributions={'RFclf__min_samples_leaf': [1, 2, 3, 4, 8, 10, 12], 'RFclf__max_features': ['auto', 'sqrt', 'log2'], 'RFclf__bootstrap': [0, 1], 'RFclf__max_depth': [4, 5, 6, 8, 12, None], 'RFclf__criterion': ['gini', 'entropy'], 'RFclf__min_samples_split': [3, 5, 7], 'RFclf__n_estimators': [50, 75, 100, 130, 150, 180]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_

## TEST FOR ONE SET OF PARAMETERS

RandomForestClassifier(bootstrap=0, class_weight=None, criterion='entropy',
            max_depth=12, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [13]:
#TEST WITH RANDOM PARAMETERS TO TEST FEATURES IMPORTANCE
#July
t = time.time()
Method = 'Random Forest'
print('Method:', Method)
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(bootstrap=0, criterion='entropy', max_depth=12, max_features='sqrt', 
                             min_samples_leaf=4, min_samples_split=3, n_estimators=150, n_jobs=40)

rfc.fit(X_train.values, y_train.values)

print('Elapsed time for training: %.02f sec' % (time.time() - t))
#pickle.dump(clf, open('RFmodel_1x1_scenario2_test', 'wb'))

import pickle
pickle.dump(rfc, open('RFmodel_LUCAS_'+str(biome)+'_level1-mask_all-polygons_janv-jul2018_24042020', 'wb'))

Method: Random Forest
Elapsed time for training: 41.23 sec


In [14]:
Model=rfc
print (Model)

importances = Model.feature_importances_

std = np.std([tree.feature_importances_ for tree in Model], axis=0)

indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(44):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]),X_train.columns[indices[f]])


RandomForestClassifier(bootstrap=0, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=40,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Feature ranking:
1. feature 0 (0.074120) VH_20180101
2. feature 2 (0.073029) VH_20180121
3. feature 20 (0.052349) VH_20180720
4. feature 21 (0.049794) VH_20180730
5. feature 1 (0.042506) VH_20180111
6. feature 19 (0.040007) VH_20180710
7. feature 3 (0.038395) VH_20180131
8. feature 5 (0.031169) VH_20180220
9. feature 33 (0.030943) VV_20180421
10. feature 4 (0.028349) VH_20180210
11. feature 18 (0.027240) VH_20180630
12. feature 7 (0.024927) VH_20180312
13. feature 11 (0.024236) VH_20180421
14. feature 9 (0.023860) VH_20180401
15. feature 8 (0.023179) VH_20180322
16. feature

In [11]:
rfc.grid_scores_

AttributeError: 'RandomizedSearchCV' object has no attribute 'grid_scores_'

In [17]:
#save all the models tried


In [93]:
#to import in GEE
#index_column_VHVV=lucas.columns[76:148]
#index_column_class=lucas.columns[1:4]
#print(index_column_class)
#print(index_column_VHVV)
#create a csv with all the data
#lucas_gee=lucas.loc[:,index_column_class.union(index_column_VHVV)]
#lucas_gee.to_csv('lucas_crop_grass.csv')

In [30]:
#transform in byte
#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler
#data=X_features

#scaler.data_max_=0
#scaler.data_min_=-25

#print(scaler.data_max_)
#scaler.fit(lucas_test)
#scaler.fit_transform(lucas_test)

In [59]:
#some test to avoid loading polygons csv
#goupby by class and polygon - give count of pixels per polygons
#test=pd_level2.groupby(['ClassL2','POINT_I'])['ClassL2','POINT_I'].count()
#print(test.head())
#how to count the number of polygon per class? level 0?
#lucasgroup=pd_level2.groupby(['ClassL2','POINT_I'])['POINT_I'].size()
#print(lucasgroup.head())

In [ ]:
#implement with the biome class for the main crops we are interested
#+the 